In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['AST']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('aspartate aminotransferase', 1739),
 ('aspartate transaminase', 457),
 ('astaxanthin', 111),
 ('aspartate amino transferase', 56),
 ('astragaloside', 21),
 ('antimicrobial susceptibility testing', 21),
 ('aspartate aminotransaminase', 20),
 ('aspartate', 18),
 ('astressin', 16),
 ('astrocytoma', 11),
 ('aspartate transferase', 10),
 ('astragaloside iv', 10),
 ('androgen suppression therapy', 8),
 ('asparate aminotransferase', 8),
 ('acute stent thrombosis', 6),
 ('antibiotic susceptibility testing', 6),
 ('anti scarring treatment', 5),
 ('astragalin', 5),
 ('activities school transport', 5),
 ('astemizole', 5),
 ('astrocytes', 5),
 ('total astragalus saponins', 5),
 ('astilbin', 4),
 ('average survival time', 4),
 ('allatostatin', 4),
 ('attentional set shifting testing', 4),
 ('that astragalus saponins', 4),
 ('aspartate aminotrasferase', 4),
 ('aminotransferase activities', 4),
 ('artesunate', 4),
 ('aspartate transminase', 3),
 ('antenatal steroids therapy', 3),
 ('antisense tran

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-28 14:52:29] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'activities school transport': 'ungrounded',
  'acute stent thrombosis': 'ungrounded',
  'allatostatin': 'MESH:C076523',
  'aminotransferase activities': 'FPLX:GOT',
  'androgen suppression therapy': 'ungrounded',
  'antenatal steroids therapy': 'ungrounded',
  'anti scarring treatment': 'ungrounded',
  'antibiotic susceptibility testing': 'ungrounded',
  'antimicrobial susceptibility testing': 'ungrounded',
  'antisaccade task': 'ungrounded',
  'antisense transcript': 'ungrounded',
  'artesunate': 'CHEBI:CHEBI:63918',
  'asparatate aminotransferase': 'FPLX:GOT',
  'asparate aminotransferase': 'FPLX:GOT',
  'aspartate': 'FPLX:GOT',
  'aspartate amino transaminase': 'FPLX:GOT',
  'aspartate amino transferase': 'FPLX:GOT',
  'aspartate aminotransaminase': 'FPLX:GOT',
  'aspartate aminotransferase': 'FPLX:GOT',
  'aspartate aminotrasferase': 'FPLX:GOT',
  'aspartate transaminase': 'FPLX:GOT',
  'aspartate transferase': 'FPLX:GOT',
  'aspartate transminase': 'FPLX:GOT',
  'astaxanthin': 

In [6]:
grounding_map, names, pos_labels = [{'activities school transport': 'ungrounded',
  'acute stent thrombosis': 'ungrounded',
  'allatostatin': 'MESH:C076523',
  'aminotransferase activities': 'FPLX:GOT',
  'androgen suppression therapy': 'ungrounded',
  'antenatal steroids therapy': 'ungrounded',
  'anti scarring treatment': 'ungrounded',
  'antibiotic susceptibility testing': 'ungrounded',
  'antimicrobial susceptibility testing': 'ungrounded',
  'antisaccade task': 'ungrounded',
  'antisense transcript': 'ungrounded',
  'artesunate': 'CHEBI:CHEBI:63918',
  'asparatate aminotransferase': 'FPLX:GOT',
  'asparate aminotransferase': 'FPLX:GOT',
  'aspartate': 'FPLX:GOT',
  'aspartate amino transaminase': 'FPLX:GOT',
  'aspartate amino transferase': 'FPLX:GOT',
  'aspartate aminotransaminase': 'FPLX:GOT',
  'aspartate aminotransferase': 'FPLX:GOT',
  'aspartate aminotrasferase': 'FPLX:GOT',
  'aspartate transaminase': 'FPLX:GOT',
  'aspartate transferase': 'FPLX:GOT',
  'aspartate transminase': 'FPLX:GOT',
  'astaxanthin': 'CHEBI:CHEBI:40968',
  'astemizole': 'CHEBI:CHEBI:2896',
  'astilbin': 'CHEBI:CHEBI:38200',
  'astragalin': 'MESH:C001579',
  'astragaloside': 'CHEBI:CHEBI:65457',
  'astragaloside iv': 'CHEBI:CHEBI:65457',
  'astragalus membranaceus': 'ungrounded',
  'astressin': 'CHEBI:CHEBI:76649',
  'astrocytes': 'MESH:D001253',
  'astrocytoma': 'MESH:D001254',
  'attentional set shifting testing': 'ungrounded',
  'average survival time': 'ungrounded',
  'that astragalus saponins': 'ungrounded',
  'total astragalus saponins': 'ungrounded'},
 {'MESH:C076523': 'allatostatin',
  'FPLX:GOT': 'GOT',
  'CHEBI:CHEBI:63918': 'artesunate',
  'CHEBI:CHEBI:40968': 'astaxanthin',
  'CHEBI:CHEBI:2896': 'astemizole',
  'CHEBI:CHEBI:38200': 'astilbin',
  'MESH:C001579': 'astragalin',
  'CHEBI:CHEBI:65457': 'astragaloside IV',
  'CHEBI:CHEBI:76649': 'astressin',
  'MESH:D001253': 'Astrocytes',
  'MESH:D001254': 'Astrocytoma'},
 ['CHEBI:CHEBI:40968', 'FPLX:GOT']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {'HGNC:10933': ['SLC17A5', ['AST']]}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:10933', 'HGNC:10933', 998)]

In [15]:
intersection2

[('FPLX:GOT', 'HGNC:10933', 0),
 ('MESH:C076523', 'HGNC:10933', 0),
 ('ungrounded', 'HGNC:10933', 0),
 ('MESH:D001253', 'HGNC:10933', 0),
 ('CHEBI:CHEBI:40968', 'HGNC:10933', 0),
 ('CHEBI:CHEBI:76649', 'HGNC:10933', 0),
 ('CHEBI:CHEBI:65457', 'HGNC:10933', 0),
 ('CHEBI:CHEBI:2896', 'HGNC:10933', 0),
 ('CHEBI:CHEBI:63918', 'HGNC:10933', 0),
 ('MESH:C001579', 'HGNC:10933', 0),
 ('CHEBI:CHEBI:38200', 'HGNC:10933', 0),
 ('MESH:D001254', 'HGNC:10933', 0)]

In [16]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [17]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [18]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 02:43:17] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 02:43:48] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.968261799626293 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'FPLX:GOT': 1834,
  'MESH:C076523': 4,
  'ungrounded': 60,
  'MESH:D001253': 5,
  'CHEBI:CHEBI:40968': 78,
  'CHEBI:CHEBI:76649': 5,
  'CHEBI:CHEBI:65457': 26,
  'CHEBI:CHEBI:2896': 4,
  'CHEBI:CHEBI:63918': 4,
  'MESH:C001579': 5,
  'CHEBI:CHEBI:38200': 4,
  'MESH:D001254': 2,
  'HGNC:10933': 3},
 'f1': {'mean': 0.968262, 'std': 0.00859},
 'precision': {'mean': 0.952443, 'std': 0.010563},
 'recall': {'mean': 0.985379, 'std': 0.005862},
 'CHEBI:CHEBI:63918': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D001253': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'CHEBI:CHEBI:2896': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:C001579': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'CHEBI:CHEBI:65457': {'f1': {'mean': 

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for AST

Produces the disambiguations:
	Astrocytes	MESH:D001253
	Astrocytoma	MESH:D001254
	GOT*	FPLX:GOT
	SLC17A5*	HGNC:10933
	allatostatin	MESH:C076523
	artesunate	CHEBI:CHEBI:63918
	astaxanthin*	CHEBI:CHEBI:40968
	astemizole	CHEBI:CHEBI:2896
	astilbin	CHEBI:CHEBI:38200
	astragalin	MESH:C001579
	astragaloside IV	CHEBI:CHEBI:65457
	astressin	CHEBI:CHEBI:76649

Class level metrics:
--------------------
Grounding       	Count	F1     
             GOT*	1834	 0.9805
     astaxanthin*	  78	0.71671
      Ungrounded	  60	0.76892
astragaloside IV	  26	0.34095
      Astrocytes	   5	    0.0
       astressin	   5	    0.8
      astragalin	   5	    0.0
    allatostatin	   4	    0.2
      astemizole	   4	    0.0
      artesunate	   4	    0.0
        astilbin	   4	    0.0
         SLC17A5*	   3	    0.0
     Astrocytoma	   2	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.96826
	Precision:	0.95244
	Recall:		0.98538

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)